In [44]:
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime
from dateutil.relativedelta import relativedelta
import hashlib
import uuid

In [45]:
loan = pd.read_csv('loan.csv')
account = pd.read_csv('account.csv')
client = pd.read_csv('client.csv')
district = pd.read_csv('district.csv')

In [46]:
LLP = pd.read_csv('source_data.csv')

In [47]:
loan.head(3)

,loan_id,account_id,amount,duration,payments,status,year,month,day,fulldate,location,purpose
0,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,1,home
1,4961.0,A00000019,30276,12,2523,B,2016,4,29,2016-04-29,1,debt_consolidation
2,4962.0,A00000025,30276,12,2523,A,2017,12,8,2017-12-08,1,debt_consolidation


In [48]:
# Convert the 'fulldate_x' column to datetime if it's not already
loan['fulldate'] = pd.to_datetime(loan['fulldate'])
 
# Assume the current date is 2018-12-10
current_date = datetime.strptime('2018-12-10', '%Y-%m-%d')
 
# Function to calculate the difference in months with reversed signs
def calculate_remaining_months(row):
    delta = relativedelta(current_date, row['fulldate'])
    return -(delta.years * 12 + delta.months - row['duration'])

# Apply the function to calculate the accurate remaining months with reversed signs
loan['remaining_months'] = loan.apply(calculate_remaining_months, axis=1)

In [96]:
loan

,loan_id,account_id,amount,duration,payments,status,year,month,day,fulldate,location,purpose,remaining_months
0,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,1,home,-35
1,4961.0,A00000019,30276,12,2523,B,2016,4,29,2016-04-29,1,debt_consolidation,-19
2,4962.0,A00000025,30276,12,2523,A,2017,12,8,2017-12-08,1,debt_consolidation,0
3,4967.0,A00000037,318480,60,5308,D,2018,10,14,2018-10-14,1,home,59
4,4968.0,A00000038,110736,48,2307,C,2018,4,19,2018-04-19,1,home,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,7294.0,A00011327,39168,24,1632,C,2018,9,27,2018-09-27,1,debt_consolidation,22
678,7295.0,A00011328,280440,60,4674,C,2018,7,18,2018-07-18,1,home,56
679,7304.0,A00011349,419880,60,6998,C,2015,10,29,2015-10-29,1,home,23
680,7305.0,A00011359,54024,12,4502,A,2016,8,6,2016-08-06,1,home,-16


In [50]:
account.head(3)

,account_id,district_id,frequency,parseddate,year,month,day
0,A00000001,18,Monthly Issuance,2015-03-24,2015,3,24
1,A00000002,1,Monthly Issuance,2013-02-26,2013,2,26
2,A00000003,5,Monthly Issuance,2017-07-07,2017,7,7


In [95]:
client

,client_id,sex,fulldate,day,month,year,age,social,first,middle,last,phone,email,address_1,address_2,city,state,zipcode,district_id
0,C00000001,Female,1990-12-13,13,12,1990,29,926-93-2157,Emma,Avaya,Smith,367-171-6840,emma.smith@gmail.com,387 Wellington Ave.,Unit 1,Albuquerque,NM,47246,18
1,C00000002,Male,1965-02-04,4,2,1965,54,806-94-5725,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040,1
2,C00000003,Female,1960-10-09,9,10,1960,59,614-70-9100,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162,1
3,C00000004,Male,1976-12-01,1,12,1976,43,580-20-3414,Liam,Irvin,White,951-567-8925,liam.white@gmail.com,7607 Sunnyslope Street,NaN,Indianapolis,IN,49047,5
4,C00000005,Female,1980-07-03,3,7,1980,39,536-14-5809,Sophia,Danae,Williams,428-265-1568,sophia.williams@gmail.com,755 Galvin Street,NaN,Indianapolis,IN,40852,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,C00013955,Female,1965-10-30,30,10,1965,54,830-38-3530,Adrianna,Jillian,Moore,212-205-7422,adrianna.moore@outlook.com,9405 SW. Leeton Ridge Drive,NaN,New York City,NY,10121,1
5365,C00013956,Male,1963-04-06,6,4,1963,56,773-25-4409,Aaron,Roman,Park,212-645-8820,aaron.park@gmail.com,7021 Chapel Ave.,NaN,New York City,NY,10118,1
5366,C00013968,Male,1988-04-13,13,4,1988,31,227-17-0778,Eli,Leonardo,Warner,508-465-5164,eli.warner@hotmail.com,43 Edgemont Dr.,NaN,Fall River,MA,2721,61
5367,C00013971,Female,1982-10-19,19,10,1982,37,278-77-3298,Ruth,Joy,Martin,856-636-3840,ruth.martin@gmail.com,7087 E. Pennsylvania Street,Unit 8,Danbury,CT,48475,67


In [52]:
#client['social'] = client['social'].astype(str)

#client['hashed_social'] = client['social'].apply(lambda ssn: hashlib.sha256(ssn.encode()).hexdigest() if ssn != "nan" else "NaN")

In [53]:
#client['pseudo_phone'] = client['phone'].apply(lambda x: 'XXX-XXX-' + x[-4:])

In [54]:
district.head()

,district_id,city,state_name,state_abbrev,region,division,bank_branch
0,1,New York City,New York,NY,Northeast,Middle Atlantic,Quay Manhattan
1,2,Jacksonville,Florida,FL,South,South Atlantic,Quay Detroit Central
2,3,Columbus,Ohio,OH,Midwest,East North Central,Quay Nachsville Main


In [55]:
# Function to get latitude and longitude for a city
def get_lat_long(city):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(city, country_codes="US")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Apply the function to the DataFrame
district['Latitude'], district['Longitude'] = zip(*district['city'].map(get_lat_long))

district

,district_id,city,state_name,state_abbrev,region,division,bank_branch,Latitude,Longitude
0,1,New York City,New York,NY,Northeast,Middle Atlantic,Quay Manhattan,40.712728,-74.006015
1,2,Jacksonville,Florida,FL,South,South Atlantic,Quay Detroit Central,30.332184,-81.655651
2,3,Columbus,Ohio,OH,Midwest,East North Central,Quay Nachsville Main,39.962260,-83.000707
3,4,Charlotte,North Carolina,NC,South,South Atlantic,Quay Portland,35.227209,-80.843083
4,5,Indianapolis,Indiana,IN,Northeast,East North Central,Oklahoma City Quay,39.768333,-86.158350
...,...,...,...,...,...,...,...,...,...
72,73,New Britain,Connecticut,CT,Northeast,New England,Rochester Quay,41.661210,-72.779542
73,74,Chicago,Illinois,IL,Northeast,East North Central,Quay Yonkers,41.875562,-87.624421
74,75,Rochester,New York,NY,Northeast,Middle Atlantic,Quay Syracuse branch,43.157285,-77.615214
75,76,Yonkers,New York,NY,Northeast,Middle Atlantic,Washinton State Quay,40.931210,-73.898747


## Merge tables

In [106]:
LA = loan.merge(account, how='inner', on='account_id')

In [97]:
LA

,loan_id,account_id,amount,duration,payments,status,year_x,month_x,day_x,fulldate,location,purpose,remaining_months,district_id,frequency,parseddate,year_y,month_y,day_y
0,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,1,home,-35,1,Monthly Issuance,2013-02-26,2013,2,26
1,4961.0,A00000019,30276,12,2523,B,2016,4,29,2016-04-29,1,debt_consolidation,-19,21,Monthly Issuance,2015-04-07,2015,4,7
2,4962.0,A00000025,30276,12,2523,A,2017,12,8,2017-12-08,1,debt_consolidation,0,68,Monthly Issuance,2016-07-28,2016,7,28
3,4967.0,A00000037,318480,60,5308,D,2018,10,14,2018-10-14,1,home,59,20,Monthly Issuance,2017-08-18,2017,8,18
4,4968.0,A00000038,110736,48,2307,C,2018,4,19,2018-04-19,1,home,41,19,Weekly Issuance,2017-08-08,2017,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,7294.0,A00011327,39168,24,1632,C,2018,9,27,2018-09-27,1,debt_consolidation,22,7,Monthly Issuance,2017-10-15,2017,10,15
678,7295.0,A00011328,280440,60,4674,C,2018,7,18,2018-07-18,1,home,56,54,Monthly Issuance,2016-11-05,2016,11,5
679,7304.0,A00011349,419880,60,6998,C,2015,10,29,2015-10-29,1,home,23,1,Weekly Issuance,2015-05-26,2015,5,26
680,7305.0,A00011359,54024,12,4502,A,2016,8,6,2016-08-06,1,home,-16,61,Monthly Issuance,2014-10-01,2014,10,1


In [107]:
LAD = LA.merge(district, how='inner', on='district_id')

In [108]:
LAD

,loan_id,account_id,amount,duration,payments,status,year_x,month_x,day_x,fulldate,...,month_y,day_y,city,state_name,state_abbrev,region,division,bank_branch,Latitude,Longitude
0,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,2,26,New York City,New York,NY,Northeast,Middle Atlantic,Quay Manhattan,40.712728,-74.006015
1,5060.0,A00000426,252060,60,4201,D,2014,7,19,2014-07-19,...,11,29,New York City,New York,NY,Northeast,Middle Atlantic,Quay Manhattan,40.712728,-74.006015
2,5117.0,A00000718,76944,12,6412,A,2017,8,20,2017-08-20,...,3,3,New York City,New York,NY,Northeast,Middle Atlantic,Quay Manhattan,40.712728,-74.006015
3,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,5,9,New York City,New York,NY,Northeast,Middle Atlantic,Quay Manhattan,40.712728,-74.006015
4,5332.0,A00001869,37944,36,1054,C,2018,1,29,2018-01-29,...,6,14,New York City,New York,NY,Northeast,Middle Atlantic,Quay Manhattan,40.712728,-74.006015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,6605.0,A00007890,79344,48,1653,C,2017,10,14,2017-10-14,...,3,26,Minneapolis,Minnesota,MN,Midwest,West North Central,Quay Columbus Main St,44.977300,-93.265469
678,6729.0,A00008519,74688,12,6224,A,2014,8,11,2014-08-11,...,4,2,Minneapolis,Minnesota,MN,Midwest,West North Central,Quay Columbus Main St,44.977300,-93.265469
679,7115.0,A00010351,88704,48,1848,C,2017,3,4,2017-03-04,...,5,4,Minneapolis,Minnesota,MN,Midwest,West North Central,Quay Columbus Main St,44.977300,-93.265469
680,7067.0,A00010134,136368,24,5682,C,2018,1,11,2018-01-11,...,4,26,Virginia Beach,Virginia,VA,South,South Atlantic,Newark Quay,36.852984,-75.977418


In [160]:
full = LAD.merge(client, how='inner', on='district_id')

In [161]:
full

,loan_id,account_id,amount,duration,payments,status,year_x,month_x,day_x,fulldate_x,...,first,middle,last,phone,email,address_1,address_2,city_y,state,zipcode
0,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040
1,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162
2,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Madison,Kimberlyn,Martinez,212-093-8170,madison.martinez@gmail.com,7062 Temple Court,Unit 1B,New York City,NY,10282
3,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Daniel,Isa,Hall,212-803-8384,daniel.hall67@gmail.com,30 West Westport Dr.,NaN,New York City,NY,10009
4,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Aiden,Jamil,Young,212-751-6276,aiden.young9@gmail.com,125 North Holly Dr.,NaN,New York City,NY,10158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99235,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Devonte,Benedict,Bacon,825-948-4689,devonte.bacon@outlook.com,687 North Armstrong Road,NaN,Virginia Beach,VA,46258
99236,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Eloise,Natalie,Jensen,701-011-9415,eloise.jensen@gmail.com,8383 Halifax St.,NaN,Virginia Beach,VA,47097
99237,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Eleazar,Bilal,Crowe,983-787-5445,eleazar.crowe@gmail.com,496 Bridge St.,NaN,Virginia Beach,VA,45305
99238,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Justin,Wyatt,Pearson,381-444-5173,justin.pearson@outlook.com,497 Westport Rd.,NaN,Virginia Beach,VA,40487


## Narrow down 'home' customers with: 
* Loan status of A or C
* Payments over 25% percentile of LLP (5256.095093)

In [162]:
home = full[full['purpose'] == 'home']
home

,loan_id,account_id,amount,duration,payments,status,year_x,month_x,day_x,fulldate_x,...,first,middle,last,phone,email,address_1,address_2,city_y,state,zipcode
0,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040
1,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162
2,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Madison,Kimberlyn,Martinez,212-093-8170,madison.martinez@gmail.com,7062 Temple Court,Unit 1B,New York City,NY,10282
3,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Daniel,Isa,Hall,212-803-8384,daniel.hall67@gmail.com,30 West Westport Dr.,NaN,New York City,NY,10009
4,4959.0,A00000002,80952,24,3373,A,2014,1,5,2014-01-05,...,Aiden,Jamil,Young,212-751-6276,aiden.young9@gmail.com,125 North Holly Dr.,NaN,New York City,NY,10158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99235,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Devonte,Benedict,Bacon,825-948-4689,devonte.bacon@outlook.com,687 North Armstrong Road,NaN,Virginia Beach,VA,46258
99236,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Eloise,Natalie,Jensen,701-011-9415,eloise.jensen@gmail.com,8383 Halifax St.,NaN,Virginia Beach,VA,47097
99237,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Eleazar,Bilal,Crowe,983-787-5445,eleazar.crowe@gmail.com,496 Bridge St.,NaN,Virginia Beach,VA,45305
99238,7122.0,A00010365,260640,36,7240,D,2017,1,4,2017-01-04,...,Justin,Wyatt,Pearson,381-444-5173,justin.pearson@outlook.com,497 Westport Rd.,NaN,Virginia Beach,VA,40487


In [163]:
C = home[home['status'] == 'C']

In [164]:
A = home[home['status'] == 'A']

In [165]:
home_AC = pd.concat([A, C])

In [166]:
home_AC.columns

Index(['loan_id', 'account_id', 'amount', 'duration', 'payments', 'status',
       'year_x', 'month_x', 'day_x', 'fulldate_x', 'location', 'purpose',
       'remaining_months', 'district_id', 'frequency', 'parseddate', 'year_y',
       'month_y', 'day_y', 'city_x', 'state_name', 'state_abbrev', 'region',
       'division', 'bank_branch', 'Latitude', 'Longitude', 'client_id', 'sex',
       'fulldate_y', 'day', 'month', 'year', 'age', 'social', 'first',
       'middle', 'last', 'phone', 'email', 'address_1', 'address_2', 'city_y',
       'state', 'zipcode'],
      dtype='object')

In [167]:
plat_home1 = home_AC[home_AC['payments'] > 5256.095093]

In [168]:
plat_home1

,loan_id,account_id,amount,duration,payments,status,year_x,month_x,day_x,fulldate_x,...,first,middle,last,phone,email,address_1,address_2,city_y,state,zipcode
1326,5117.0,A00000718,76944,12,6412,A,2017,8,20,2017-08-20,...,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040
1327,5117.0,A00000718,76944,12,6412,A,2017,8,20,2017-08-20,...,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162
1328,5117.0,A00000718,76944,12,6412,A,2017,8,20,2017-08-20,...,Madison,Kimberlyn,Martinez,212-093-8170,madison.martinez@gmail.com,7062 Temple Court,Unit 1B,New York City,NY,10282
1329,5117.0,A00000718,76944,12,6412,A,2017,8,20,2017-08-20,...,Daniel,Isa,Hall,212-803-8384,daniel.hall67@gmail.com,30 West Westport Dr.,NaN,New York City,NY,10009
1330,5117.0,A00000718,76944,12,6412,A,2017,8,20,2017-08-20,...,Aiden,Jamil,Young,212-751-6276,aiden.young9@gmail.com,125 North Holly Dr.,NaN,New York City,NY,10158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99184,7067.0,A00010134,136368,24,5682,C,2018,1,11,2018-01-11,...,Devonte,Benedict,Bacon,825-948-4689,devonte.bacon@outlook.com,687 North Armstrong Road,NaN,Virginia Beach,VA,46258
99185,7067.0,A00010134,136368,24,5682,C,2018,1,11,2018-01-11,...,Eloise,Natalie,Jensen,701-011-9415,eloise.jensen@gmail.com,8383 Halifax St.,NaN,Virginia Beach,VA,47097
99186,7067.0,A00010134,136368,24,5682,C,2018,1,11,2018-01-11,...,Eleazar,Bilal,Crowe,983-787-5445,eleazar.crowe@gmail.com,496 Bridge St.,NaN,Virginia Beach,VA,45305
99187,7067.0,A00010134,136368,24,5682,C,2018,1,11,2018-01-11,...,Justin,Wyatt,Pearson,381-444-5173,justin.pearson@outlook.com,497 Westport Rd.,NaN,Virginia Beach,VA,40487


In [169]:
plat_home1.columns

Index(['loan_id', 'account_id', 'amount', 'duration', 'payments', 'status',
       'year_x', 'month_x', 'day_x', 'fulldate_x', 'location', 'purpose',
       'remaining_months', 'district_id', 'frequency', 'parseddate', 'year_y',
       'month_y', 'day_y', 'city_x', 'state_name', 'state_abbrev', 'region',
       'division', 'bank_branch', 'Latitude', 'Longitude', 'client_id', 'sex',
       'fulldate_y', 'day', 'month', 'year', 'age', 'social', 'first',
       'middle', 'last', 'phone', 'email', 'address_1', 'address_2', 'city_y',
       'state', 'zipcode'],
      dtype='object')

In [182]:
plat_home1.to_csv('plat_home1.csv')

## Narrow down 'home improvement' customers with:
* Loan status of A or C
* Payment over the minimum payment on LLP (1900.780029)
* Loan amount over $40000

In [171]:
home_imp = full[full['purpose'] == 'home_improvement']
home_imp.head(3)

,loan_id,account_id,amount,duration,payments,status,year_x,month_x,day_x,fulldate_x,...,first,middle,last,phone,email,address_1,address_2,city_y,state,zipcode
1989,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040
1990,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162
1991,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Madison,Kimberlyn,Martinez,212-093-8170,madison.martinez@gmail.com,7062 Temple Court,Unit 1B,New York City,NY,10282


In [172]:
HI_C = home_imp[home_imp['status'] == 'C']

In [173]:
HI_A = home_imp[home_imp['status'] == 'A']

In [174]:
home_imp_AC = pd.concat([HI_A, HI_C])

In [175]:
home_imp_AC.columns

Index(['loan_id', 'account_id', 'amount', 'duration', 'payments', 'status',
       'year_x', 'month_x', 'day_x', 'fulldate_x', 'location', 'purpose',
       'remaining_months', 'district_id', 'frequency', 'parseddate', 'year_y',
       'month_y', 'day_y', 'city_x', 'state_name', 'state_abbrev', 'region',
       'division', 'bank_branch', 'Latitude', 'Longitude', 'client_id', 'sex',
       'fulldate_y', 'day', 'month', 'year', 'age', 'social', 'first',
       'middle', 'last', 'phone', 'email', 'address_1', 'address_2', 'city_y',
       'state', 'zipcode'],
      dtype='object')

In [176]:
home_imp_AC1 = home_imp_AC[home_imp_AC['payments'] > 1900.780029]

In [177]:
home_imp_AC2 = (home_imp_AC[home_imp_AC['amount'] > 40000])

In [178]:
plat_home_imp1 = pd.concat([home_imp_AC1, home_imp_AC2])

In [179]:
plat_home_imp1

,loan_id,account_id,amount,duration,payments,status,year_x,month_x,day_x,fulldate_x,...,first,middle,last,phone,email,address_1,address_2,city_y,state,zipcode
1989,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040
1990,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162
1991,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Madison,Kimberlyn,Martinez,212-093-8170,madison.martinez@gmail.com,7062 Temple Court,Unit 1B,New York City,NY,10282
1992,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Daniel,Isa,Hall,212-803-8384,daniel.hall67@gmail.com,30 West Westport Dr.,NaN,New York City,NY,10009
1993,5285.0,A00001603,78936,12,6578,A,2014,2,6,2014-02-06,...,Aiden,Jamil,Young,212-751-6276,aiden.young9@gmail.com,125 North Holly Dr.,NaN,New York City,NY,10158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98560,6474.0,A00007190,63360,60,1056,C,2015,6,6,2015-06-06,...,Miriam,Genesis,Delgado,955-474-2346,miriam.delgado@hotmail.com,9883 Studebaker Ave.,NaN,Burlington,VT,39929
98561,6474.0,A00007190,63360,60,1056,C,2015,6,6,2015-06-06,...,Ryan,Antony,Rogers,817-770-7405,ryan.rogers@outlook.com,256 Piper Street,NaN,Burlington,VT,47754
98562,6474.0,A00007190,63360,60,1056,C,2015,6,6,2015-06-06,...,Jason,Shamar,Graham,175-084-6578,jason.graham@outlook.com,68 Newcastle St.,NaN,Burlington,VT,32345
98563,6474.0,A00007190,63360,60,1056,C,2015,6,6,2015-06-06,...,Sebastian,Cecil,Jimenez,485-783-8026,sebastian.jimenez@outlook.com,7897 Orange Circle,NaN,Burlington,VT,36125


In [180]:
plat_home_imp1.columns

Index(['loan_id', 'account_id', 'amount', 'duration', 'payments', 'status',
       'year_x', 'month_x', 'day_x', 'fulldate_x', 'location', 'purpose',
       'remaining_months', 'district_id', 'frequency', 'parseddate', 'year_y',
       'month_y', 'day_y', 'city_x', 'state_name', 'state_abbrev', 'region',
       'division', 'bank_branch', 'Latitude', 'Longitude', 'client_id', 'sex',
       'fulldate_y', 'day', 'month', 'year', 'age', 'social', 'first',
       'middle', 'last', 'phone', 'email', 'address_1', 'address_2', 'city_y',
       'state', 'zipcode'],
      dtype='object')

In [183]:
plat_home_imp1.to_csv('plat_home_imp1.csv')

## Overview of LLP table

In [ ]:
LLP.describe()